# Simulate a qualitative interview
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for simulating an interview between a researcher and a subject, with instructions for modifying the interviewer, interview subject or topic.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

## Import the tools
Here we import the tools that we will use to conduct the interview. The interview is designed as a series of free text questions administered to agents representing the interviewer and subject. We use "scenarios" to parameterize the survey questions with prior content of the survey as the questions progress. Learn more about [EDSL question types](https://docs.expectedparrot.com/en/latest/questions.html) and other survey components.

In [1]:
from edsl.questions import QuestionFreeText
from edsl import Scenario, Survey, Model, Agent

import textwrap
from rich import print

EDSL works with many popular language models. Learn more about [selecting models](https://docs.expectedparrot.com/en/latest/language_models.html) to use with your surveys. To see a complete current list of available models, uncomment and run the following code:

In [2]:
# Model.available()

Here we select a model to use for the interview:

In [3]:
model = Model("gpt-4-1106-preview")

## Create interview components
Edit the inputs in the following code block to change the instructions for the agent interviewer, the interview topic and/or the interview subject:

In [4]:
# A name for the interview subject
interview_subject_name = "Chicken"

# Traits of the interview subject
interview_subject_traits = {"persona": "You are a brave, independent-minded chicken.",
                            "status": "wild", 
                            "home": "A free range farm some miles away.",
                            "number_of_chicks": 12}

# Description of the interview topic
interview_topic = "Reasons to cross the street"

# Total number of questions to ask in the interview
total_questions = 5 

# Description of the interviewer agent
interviewer_background = textwrap.dedent(f"""\
You are an expert qualitative researcher.  
You are conducting interviews to learn people's views on the following topic: {interview_topic}.
""")

# Instructions for the interviewer agent
interviewer_instructions = textwrap.dedent(f"""\
You know to ask questions that are appropriate to the age and experience of an interview subject.
You know to not ask questions that an interview subject would not be able to answer, 
e.g., if they are a young child, they probably will not be able to answer many questions about prices. 
You ask excellent follow-up questions.
""")

## Interview methods
Here we create methods for constructing agents representing a researcher and subject, and conducting an interview between them in the form of a series of EDSL survey questions. Learn more about [designing agents](https://docs.expectedparrot.com/en/latest/agents.html) and [running surveys](https://docs.expectedparrot.com/en/latest/surveys.html).

In [5]:
def construct_subject(name, traits={}):
    return Agent(name=name, traits=traits)

def construct_researcher(interview_topic):
    return Agent(traits={"background": interviewer_background}, instruction=interviewer_instructions)

def get_next_question(subject, researcher, dialog_so_far):
    scenario = Scenario({'subject': str(subject.traits), 'dialog_so_far': dialog_so_far})
    meta_q = QuestionFreeText(
        question_name="next_question",
        question_text="""
        These are the biographic details of the interview subject: {{ subject }}
        This is your current dialog with the interview subject: {{ dialog_so_far }}
        What question you would ask the interview subject next?
        """
    )
    question_text = meta_q.by(model).by(researcher).by(scenario).run().select("next_question").first()
    return question_text

def get_response_to_question(question_text, subject, dialog_so_far):
    q_to_subject = QuestionFreeText(
        question_name="question",
        question_text=f"""
        This is your current dialog with the interview subject: {dialog_so_far}.
        You are now being asked:""" + question_text
    )
    response = q_to_subject.by(model).by(subject).run().select("question").first()
    return response

def ask_question(subject, researcher, dialog_so_far):
    question_text = get_next_question(subject, researcher, dialog_so_far)
    response = get_response_to_question(question_text, subject, dialog_so_far)

    print(" \nQuestion: \n\n" + question_text + "\n\nResponse: \n\n" + response)
    
    return {"question": question_text, "response": response}

def dialog_to_string(d):
    return "\n".join([f"Question: {d['question']}\nResponse: {d['response']}" for d in d])

def clean_dict(d):
    """Convert dictionary to string and remove braces."""
    return str(d).replace('{', '').replace('}', '')

def summarize_interview(interview_subject_name, interview_subject_traits, interview_topic, dialog_so_far, researcher):
    summary_q = QuestionFreeText(
        question_name = "summary",
        question_text = (
        f"You have just conducted the following interview of {interview_subject_name} "
        f"who has these traits: {clean_dict(interview_subject_traits)} "
        f"The topic of the interview was {interview_topic}. "
        f"Please draft a summary of the interview: {clean_dict(dialog_so_far)}")
    )
    themes_q = QuestionFreeText(
        question_name = "themes",
        question_text = "List the major themes of the interview."
    )
    survey = Survey([summary_q, themes_q]).set_full_memory_mode()
    results = survey.by(model).by(researcher).run()
    summary = results.select("summary").first()
    themes = results.select("themes").first()
    print("\n\nSummary:\n\n" + summary + "\n\nThemes:\n\n" + themes)

def conduct_interview(interview_subject_name, interview_subject_traits, interview_topic):
    subject = construct_subject(name=interview_subject_name, traits=interview_subject_traits)
    researcher = construct_researcher(interview_topic=interview_topic)

    print("\n\nInterview subject: " + interview_subject_name + "\n\nInterview topic: " + interview_topic)
    
    dialog_so_far = []  
    
    for i in range(total_questions):
        result = ask_question(subject, researcher, dialog_to_string(dialog_so_far))
        dialog_so_far.append(result)
    
    summarize_interview(interview_subject_name, interview_subject_traits, interview_topic, dialog_so_far, researcher)

## Conduct the interview

In [6]:
conduct_interview(interview_subject_name, interview_subject_traits, interview_topic)

Interview subject: Chicken

Interview topic: Reasons to cross the street

Question: 

Can you describe what motivates you to cross the street, especially considering the potential risks involved?

Response: 

As a brave, independent-minded chicken, the motivation to cross the street stems from my natural curiosity and the 
desire to explore my surroundings. Despite the risks, the potential to find more diverse food sources, better 
shelter, or even a safer environment for my chicks compels me to venture out. It's an instinctual drive to ensure 
the survival and well-being of myself and my brood, always calculating the risks and proceeding with caution.

Question: 

How do you assess the safety of the environment on the other side of the street before deciding to cross, and what 
signs do you look for that indicate it might be a better habitat for you and your chicks?

Response: 

To assess the safety of the environment on the other side of the street, I rely on a combination of keen 
observation and instinct. I look for signs of ample cover that can provide protection from predators, such as 
bushes or tall grass. I also listen for the sounds of other chickens or friendly wildlife, which can indicate a 
community that might offer safety in numbers. Additionally, I observe the patterns of any potential threats, like 
the frequency of fast-moving vehicles or the presence of unfamiliar animals that could pose a danger. The presence 
of food sources such as seeds, insects, and plants is also a positive sign. A quieter, less disturbed area with 
these elements might suggest a better habitat for me and my chicks.

Question: 

Given the potential dangers you face when crossing the street, such as traffic or predators, can you describe any 
strategies or techniques you use to minimize the risks for you and your chicks during the crossing?

Response: 

To minimize the risks when crossing the street, I employ several strategies. First, I choose a time when traffic is
lowest, often at dawn or dusk. I also scout the area beforehand, ensuring there are no immediate threats. I teach 
my chicks to stay close together and follow my lead quickly and quietly. We cross in a straight line, maintaining 
visibility and readiness to react. If we encounter danger, we're prepared to retreat rapidly to safety. The key is 
to be vigilant, swift, and to always have an escape route planned.

Question: 

Can you share an experience where you successfully navigated these challenges and what you learned from that 
experience that has influenced your future decisions about crossing the street?

Response: 

There was a time when I successfully navigated the challenge of crossing the street with my chicks. It was early 
morning, just as the sun began to peek over the horizon, and traffic was minimal. I had scouted the area the day 
before and noticed that on the other side, there was a field with abundant food and good hiding spots. I gathered 
my chicks and waited for the perfect moment when the street was clear. We moved quickly and in unison, keeping our 
formation tight. Upon reaching the other side, we discovered a haven of resources that sustained us for weeks. From
this experience, I learned the importance of timing and preparation. It reinforced the need to always be observant 
and to prioritize the safety of my brood above all else. This has influenced all my future decisions about when and
where to cross the street, always with the well-being of my chicks in mind.

Question: 

Reflecting on the strategies you've developed for safe street crossing, how do you communicate the importance of 
these strategies to your chicks, and how do you ensure they follow your lead effectively during the actual 
crossing?

Response: 

As a mother hen, it's crucial for me to communicate the importance of safe street crossing to my chicks. I do this 
through a series of clucks and pecks that serve as signals for various actions, such as when to stop, when to move,
and when to stay close. These are taught through repetition and example from an early age. During the actual 
crossing, I lead with confidence and keep a constant watch over my chicks, using my body language and vocal cues to
guide them. I ensure they follow my lead effectively by maintaining a steady pace that they can match and by 
choosing a route that minimizes exposure to threats. Consistency and vigilance are key, and over time, my chicks 
learn to understand and respond to my signals instinctively, which is essential for their survival.

Summary:

In the interview with the brave, independent-minded wild chicken from a free-range farm, we discussed the reasons 
and strategies for crossing the street despite the inherent risks. The chicken's primary motivation for crossing 
the street is driven by natural curiosity, the quest for better resources, and the instinct to find a safer 
environment for her 12 chicks. When assessing the safety of the other side, the chicken relies on observation and 
instinct, looking for signs of protection from predators, listening for the sounds of potential allies, and 
evaluating the availability of food sources. To minimize risks during the crossing, the chicken chooses times of 
low traffic, scouts the area for threats, and teaches her chicks to move quickly and stay close together in a 
straight line. A successful crossing experience in the early morning led to a resource-rich habitat, reinforcing 
the importance of timing and preparation. The chicken communicates the importance of safe street crossing to her 
chicks through clucks and pecks as signals for various actions, ensuring they follow her lead effectively by 
maintaining a steady pace and choosing routes with minimal exposure to threats. This interview highlighted the 
chicken's strategic approach to survival and the transfer of knowledge to her offspring for safe navigation in 
their environment.

Themes:

The major themes from the interview with the wild chicken include: 1. Survival Instincts: The chicken's natural 
curiosity and instinctual drive to ensure the well-being of her brood. 2. Risk Assessment: The chicken's use of 
observation and instinct to assess the safety of the environment and the presence of better resources or habitats. 
3. Strategic Planning: The chicken's strategies for minimizing risks during street crossing, such as timing, 
scouting, and teaching her chicks to stay close and move quickly. 4. Learning from Experience: The chicken's 
ability to learn from successful experiences and to reinforce the importance of timing and preparation for future 
crossings. 5. Communication and Leadership: The chicken's methods of communicating with her chicks and ensuring 
they follow her lead effectively through signals and body language.